In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error
from ipywidgets import interactive, IntSlider
import IPython.display as display

#données 

cf = pd.read_csv("data/CF_1d.csv", index_col = "Date", parse_dates = True)
ta = pd.read_csv("data/TA_1d.csv", index_col = "Date", parse_dates = True)
tp = pd.read_csv("data/TP_1d.csv", index_col = "Date", parse_dates = True)

cf_FR = cf[["FR"]]
ta_FR = ta[ta.columns[ta.columns.str.startswith("FR")]]
tp_FR = tp[tp.columns[tp.columns.str.startswith("FR")]]


#window optimization 
def window_optimization(min_window=1, max_window=180, step=2, search_lag_max=10, threshold=0.15):
    
    windows = range(min_window, max_window + 1, step)
    results_matrix = pd.DataFrame(index=tp_norm.columns, columns=windows)
    
    for w in windows:
        tp_rolled = tp_norm.rolling(window=w).sum()
        max_corrs_for_window = pd.Series([-1.0]*len(tp_norm.columns), index=tp_norm.columns)
        
        for lag in range(search_lag_max + 1):
            tp_shifted = tp_rolled.shift(lag)
            current_corrs = tp_shifted.corrwith(cf_mean)
            max_corrs_for_window = np.maximum(max_corrs_for_window, current_corrs)
        
        results_matrix[w] = max_corrs_for_window

    results_matrix = results_matrix.astype(float)

    best_windows = results_matrix.idxmax(axis=1)
    best_values = results_matrix.max(axis=1)
    
    optimal_df = pd.DataFrame({
        "Optimal Window": best_windows,
        "Max Correlation": best_values
    }).sort_values(by="Max Correlation", ascending=False)
    
    return optimal_df
res_tp = window_optimization()

#data uptdate - rolling
for name, column in tp_FR.items():
    if res_tp.loc[name, "Max Correlation"] < 0.45:
        tp_FR.drop(name, axis = 1)
    else :
        tp_FR[name] = tp_FR[name].rolling(window = res_tp.loc[name, "Optimal Window"], min_periods = 1).sum()

# Features & Target
X = pd.merge(ta_FR.rolling(window =14, min_periods = 1).mean(), tp_FR, left_index = True, right_index = True, suffixes = ("_TA", "_TP"))
y = cf_FR


# Séparation des données d'entrainement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=365, shuffle=False)

# Normalisation
scaler = StandardScaler().set_output(transform="pandas")
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

X_train_std["cos"] = np.cos(X_train_std.index.dayofyear * 2 * np.pi/365)
X_train_std["sin"] = np.sin(X_train_std.index.dayofyear * 2 * np.pi/365)

X_test_std["cos"] = np.cos(X_test_std.index.dayofyear * 2 * np.pi/365)
X_test_std["sin"] = np.sin(X_test_std.index.dayofyear * 2 * np.pi/365)

results2 = {"Actual": y_test}

from sklearn.ensemble import GradientBoostingRegressor

# Modèle Gradient Boosting
gbr = GradientBoostingRegressor(
    n_estimators=500,   # nombre d’étapes de boosting
    learning_rate=0.01, # vitesse d'apprentissage
    max_depth=4,        # profondeur des arbres faibles
    random_state=75
)

# Entraînement
gbr.fit(X_train_std, y_train)

# Prédictions
y_pred = gbr.predict(X_test_std)
y_pred = pd.Series(y_pred, index=y_test.index)

# Ajout dans results
results2["GBR"] = y_pred

# Métriques
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"R2: {r2:.06f}")
print(f"MSE: {mse:.06f}")
print()

display_result(y_test, y_pred)

NameError: name 'res_tp' is not defined